Package Installs

In [16]:
# %pip install numpy
# %pip install scipy
# %pip install torch
# %pip install torchvision

Imports

In [ ]:
import torch
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
from torch.utils.data import random_split, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

Load Data

In [16]:
dataset = MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
print(len(dataset))

60000


Data Preprocessing

In [17]:
BATCH_SIZE = 32
train_data, test_data = random_split(dataset, [51000, 9000])
print("length of Train Datasets: ", len(train_data))
print("length of Validation Datasets: ", len(test_data))

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle = True)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle = True)


length of Train Datasets:  51000
length of Validation Datasets:  9000


Model

In [18]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=5)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=5)
        self.fc1 = nn.Linear(3*3*64, 256)
        self.fc2 = nn.Linear(256, 10)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(F.max_pool2d(self.conv3(x), 2))
        x = F.dropout(x, p=0.5, training=self.training)
        x = x.view(-1, 3*3*64)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

cnn = CNN()
print(cnn)
it = iter(train_loader)
X_batch, y_batch = next(it)
print(cnn.forward(X_batch).shape)

CNN(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)
torch.Size([32, 10])


In [29]:
def fit(model, train_loader):
    optimizer = torch.optim.Adam(model.parameters())
    error = nn.CrossEntropyLoss()
    EPOCHS = 5
    model.train()
    for epoch in range(EPOCHS):
        correct = 0
        for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
            var_X_batch = Variable(X_batch).float()
            var_y_batch = Variable(y_batch)
            optimizer.zero_grad()
            output = model(var_X_batch)
            loss = error(output, var_y_batch)
            loss.backward()
            optimizer.step()

            predicted = torch.max(output.data, 1)[1]
            correct += (predicted == var_y_batch).sum()
            if batch_idx % 50 == 0:
                print('Epoch : {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Accuracy:{:.3f}%'.format(
                    epoch, batch_idx*len(X_batch), len(train_loader.dataset), 100.*batch_idx / len(train_loader), loss.data, float(correct*100) / float(BATCH_SIZE*(batch_idx+1))))

In [30]:
fit(cnn, train_loader)

Epoch : 0 [0/51000 (0%)]	Loss: 0.614433	 Accuracy:75.000%
Epoch : 0 [1600/51000 (3%)]	Loss: 0.462033	 Accuracy:84.069%
Epoch : 0 [3200/51000 (6%)]	Loss: 0.154101	 Accuracy:86.077%
Epoch : 0 [4800/51000 (9%)]	Loss: 0.263335	 Accuracy:87.169%
Epoch : 0 [6400/51000 (13%)]	Loss: 0.222539	 Accuracy:87.998%
Epoch : 0 [8000/51000 (16%)]	Loss: 0.073957	 Accuracy:88.770%
Epoch : 0 [9600/51000 (19%)]	Loss: 0.146413	 Accuracy:89.410%
Epoch : 0 [11200/51000 (22%)]	Loss: 0.058660	 Accuracy:89.939%
Epoch : 0 [12800/51000 (25%)]	Loss: 0.380743	 Accuracy:90.337%
Epoch : 0 [14400/51000 (28%)]	Loss: 0.298616	 Accuracy:90.604%
Epoch : 0 [16000/51000 (31%)]	Loss: 0.226133	 Accuracy:90.974%
Epoch : 0 [17600/51000 (35%)]	Loss: 0.171374	 Accuracy:91.306%
Epoch : 0 [19200/51000 (38%)]	Loss: 0.041673	 Accuracy:91.603%
Epoch : 0 [20800/51000 (41%)]	Loss: 0.195729	 Accuracy:91.782%
Epoch : 0 [22400/51000 (44%)]	Loss: 0.215980	 Accuracy:92.002%
Epoch : 0 [24000/51000 (47%)]	Loss: 0.135844	 Accuracy:92.173%
Epoch 

In [31]:
def evaluate(model):
    correct = 0
    for test_imgs, test_labels in test_loader:
        test_imgs = Variable(test_imgs).float()
        output = model(test_imgs)
        predicted = torch.max(output, 1)[1]
        correct += (predicted == test_labels).sum()
    print("Test accuracy:{:.3f}%".format(float(correct) / (len(test_loader)*BATCH_SIZE)))

In [32]:
evaluate(cnn)

Test accuracy:0.973%
